In [4]:
import pandas as pd
import numpy as np

#load file
sales = pd.read_csv('sales.csv')
prices = pd.read_csv('prices.csv')

In [7]:
#convert ordered_at, updated_at to datetime object

sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
prices['updated_at'] = pd.to_datetime(prices['updated_at'])

In [8]:
sales = sales.sort_values('ordered_at')
prices = prices.sort_values('updated_at')

In [11]:
# use merge_asof with direction = 'nearest' to merge sales and prices

find_price = pd.merge_asof(sales, prices, left_on = 'ordered_at', right_on = 'updated_at', 
                           by = 'product_id', direction ='nearest')


# compare ordered_at and "nearest" updated_at of each order to find the correct price

find_price['final_price'] = np.where(find_price['ordered_at'] >= find_price['updated_at'],
                                     find_price['new_price'], find_price['old_price'])

find_price

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,final_price
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,60000
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,60000
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,60000
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,60000
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,53500
...,...,...,...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,53500,52000,2018-09-17 22:59:00,52000
171,4085861,2018-09-18 20:43:00,1,53500,52000,2018-09-17 22:59:00,52000
172,4085861,2018-09-18 20:54:00,1,53500,52000,2018-09-17 22:59:00,52000
173,3954203,2018-09-18 21:26:00,1,60500,57500,2018-09-17 22:59:00,57500


In [13]:
# add a new column named 'revenue'

find_price['revenue'] = find_price['quantity_ordered'] * find_price['final_price']
find_price

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,final_price,revenue
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,60000,60000
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,60000,60000
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,60000,60000
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,60000,60000
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,53500,53500
...,...,...,...,...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,53500,52000,2018-09-17 22:59:00,52000,52000
171,4085861,2018-09-18 20:43:00,1,53500,52000,2018-09-17 22:59:00,52000,52000
172,4085861,2018-09-18 20:54:00,1,53500,52000,2018-09-17 22:59:00,52000,52000
173,3954203,2018-09-18 21:26:00,1,60500,57500,2018-09-17 22:59:00,57500,57500


In [15]:
revenue_by_product_and_price = find_price.groupby(['product_id', 'final_price'], as_index = False)['revenue'].sum()
revenue_by_product_and_price

,product_id,final_price,revenue
0,64,239000,956000
1,3954203,57500,57500
2,3954203,60000,180000
3,3954203,64000,640000
4,3998909,15500,15500
5,3998909,16500,231000
6,3998909,17000,34000
7,4085861,52000,1040000
8,4085861,53500,2140000
9,4085861,58000,2204000


In [16]:
revenue_by_product = find_price.groupby('product_id', as_index = False)['revenue'].sum()
revenue_by_product

,product_id,revenue
0,64,956000
1,3954203,877500
2,3998909,280500
3,4085861,8247500
